# Производительность документной модели на примере работы с MongoDB

<p> https://en.wikipedia.org/wiki/MongoDB
<p> https://docs.mongodb.com/manual/
<p> http://api.mongodb.com/python/current/index.html

```json
document = {
    "_id": <ObjectId>,
    "region": {
        "id": <number>,
        "name": <string>,
    },
    "year": <number>,
    "group": {
        "id": <number>,
        "name": <string>,
    },
    "category": <string>,
    "gender": <string>,
    "value": <number>,
}
```

In [1]:
import sys
import time

from pymongo import MongoClient

sys.path.append('../config/')
from config import config

In [2]:
params = config(section='mongodb')
client = MongoClient(params["host"], int(params["port"]))
db = client[params["database"]]
table = db.table

In [3]:
client.server_info()['version']

'3.6.4'

In [4]:
commands = (
    None,

    {'year': 2016},

    [
        {
            '$match': { 'year': 2016 }
        },
        {
            '$group': {
                '_id': '$group.id',
                'count': {'$sum': 1}
            }
        },
        {
            '$sort': {'_id': 1}
        }
    ],

    [
        {
            '$group': {
                '_id': '$year',
                'count': {'$sum': 1}
            }
        },
        {
            '$sort': {'_id': 1}
        }
    ],

    "group",

    {
        '_id': 0,
        'region': {
            'id': '1100',
            'name': 'Российская Федерация'
        },
        'year': 2018,
        'group': {
            'id': 14002,
            'name': 'по возрасту'
        },
        'category': '14-15',
        'gender': 'Мужчины',
        'value': 1234567890
    },
    
    {'$set': {"year": 2017, "value": 1}},
)

## 1. Собираются все документы

In [5]:
docs = [doc for doc in table.find(commands[0])]
print(len(docs))
print(docs[0])

27384
{'_id': ObjectId('5afc3e58672caf0c3490a6e3'), 'region': {'id': '1100', 'name': 'Российская Федерация'}, 'year': 2016, 'group': {'id': 14002, 'name': 'по возрасту'}, 'category': '14-15', 'gender': 'Мужчины', 'value': 13573}


In [6]:
%timeit table.find(commands[0])

47.8 µs ± 1.33 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## 2. Собираются документы за 2016 год

In [7]:
docs = [doc for doc in table.find(commands[1])]
print(len(docs))
print(docs[0])

3948
{'_id': ObjectId('5afc3e58672caf0c3490a6e3'), 'region': {'id': '1100', 'name': 'Российская Федерация'}, 'year': 2016, 'group': {'id': 14002, 'name': 'по возрасту'}, 'category': '14-15', 'gender': 'Мужчины', 'value': 13573}


In [8]:
%timeit table.find(commands[1])

46.7 µs ± 1.25 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## 3. Подсчет документов за 2016 год по группам

In [9]:
[group for group in table.aggregate(commands[2])]

[{'_id': 14002, 'count': 1128},
 {'_id': 14005, 'count': 752},
 {'_id': 14006, 'count': 1692},
 {'_id': 14007, 'count': 376}]

In [10]:
%timeit table.aggregate(commands[2])

69 ms ± 2.36 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## 4. Подсчет документов по годам

In [11]:
[i for i in table.aggregate(commands[3])]

[{'_id': 2010, 'count': 3864},
 {'_id': 2011, 'count': 3864},
 {'_id': 2012, 'count': 3864},
 {'_id': 2013, 'count': 3864},
 {'_id': 2014, 'count': 3990},
 {'_id': 2015, 'count': 3990},
 {'_id': 2016, 'count': 3948}]

In [12]:
%timeit table.aggregate(commands[3])

78.7 ms ± 2.93 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## 5. Подсчет уникальных групп

In [13]:
len(table.distinct(commands[4]))

4

In [14]:
%timeit len(table.distinct(commands[4]))

86 ms ± 10.7 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## 6. Вставка документа

In [15]:
table.insert_one(commands[5]).inserted_id

0

In [16]:
i = 1
def fun():
    global i
    commands[5]["_id"] = i
    table.insert_one(commands[5])
    i += 1

In [17]:
%timeit -n 1000 -r 5 fun()

2.48 ms ± 50.8 µs per loop (mean ± std. dev. of 5 runs, 1000 loops each)


## 7. Изменение документа

In [18]:
table.update_one({"_id": 0}, commands[6]).modified_count

1

In [19]:
i = 1
def fun():
    global i
    table.update_one({"_id": i}, commands[6])
    i += 1

In [20]:
%timeit -n 1000 -r 5 fun()

2.88 ms ± 132 µs per loop (mean ± std. dev. of 5 runs, 1000 loops each)


## 8. Удаление документа

In [21]:
table.delete_one({"_id": 0}).deleted_count

1

In [22]:
i = 1
def fun():
    global i
    table.delete_one({"_id": i})
    i += 1

In [23]:
%timeit -n 1000 -r 5 fun()

2.64 ms ± 85.3 µs per loop (mean ± std. dev. of 5 runs, 1000 loops each)


In [24]:
client.close()